In [3]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

In [4]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('./SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('./SMILE_Dataset/all/'+fname, target_size=(64, 64))) for fname in labels['fname']])
print(x)
# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded
labels

[[[[182. 182. 182.]
   [177. 177. 177.]
   [105. 105. 105.]
   ...
   [ 49.  49.  49.]
   [ 56.  56.  56.]
   [ 63.  63.  63.]]

  [[ 92.  92.  92.]
   [ 71.  71.  71.]
   [ 65.  65.  65.]
   ...
   [ 27.  27.  27.]
   [ 36.  36.  36.]
   [ 39.  39.  39.]]

  [[ 30.  30.  30.]
   [ 30.  30.  30.]
   [ 22.  22.  22.]
   ...
   [ 15.  15.  15.]
   [ 20.  20.  20.]
   [ 23.  23.  23.]]

  ...

  [[154. 154. 154.]
   [187. 187. 187.]
   [202. 202. 202.]
   ...
   [ 88.  88.  88.]
   [ 88.  88.  88.]
   [ 88.  88.  88.]]

  [[119. 119. 119.]
   [167. 167. 167.]
   [158. 158. 158.]
   ...
   [ 88.  88.  88.]
   [ 67.  67.  67.]
   [ 67.  67.  67.]]

  [[109. 109. 109.]
   [116. 116. 116.]
   [158. 158. 158.]
   ...
   [ 78.  78.  78.]
   [101. 101. 101.]
   [119. 119. 119.]]]


 [[[109. 109. 109.]
   [ 82.  82.  82.]
   [ 68.  68.  68.]
   ...
   [ 36.  36.  36.]
   [ 64.  64.  64.]
   [117. 117. 117.]]

  [[ 39.  39.  39.]
   [ 40.  40.  40.]
   [ 36.  36.  36.]
   ...
   [ 15.  15.  15.]
 

,index,fname,label
0,94,95a.JPG,neutral
1,32,33a.JPG,neutral
2,225,26b.JPG,happy
3,157,158a.JPG,neutral
4,356,157b.JPG,happy
...,...,...,...
395,299,100b.JPG,happy
396,22,23a.JPG,neutral
397,72,73a.JPG,neutral
398,15,16a.JPG,neutral


In [5]:
def pizza_model():
    
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(64, 64, 3)))
    model.add(layers.Dense(80, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [6]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = pizza_model()
    
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)   
    
    datagen.fit(X_train)

    # Secret sauce to get 3-5 % accuracy more
    # Adjust the learning rate over time. (Like we saw in class!)
    # The learning rate determines the size of the steps taken during the gradient descent process.
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Used to prevent overfitting. 
    # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 20), epochs = 10, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 20,
                              callbacks=[learning_rate_reduction])

    
    y_pred = model.predict(X_test)
    y_pred = [np.round(p[0]) for p in y_pred]
    
    print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, y_pred,output_dict=True))
    
#     if id == 3:
#         break

Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
Epoch 1/10
18/18 [==============================] - 0s 26ms/step - loss: 1.0042 - acc: 0.5500 - val_loss: 197.5574 - val_acc: 0.5000
Epoch 2/10
18/18 [==============================] - 1s 28ms/step - loss: 0.7143 - acc: 0.6028 - val_loss: 1.4423 - val_acc: 0.9250
Epoch 3/10
18/18 [==============================] - 0s 27ms/step - loss: 0.4956 - acc: 0.7806 - val_loss: 5.0159 - val_acc: 0.9750
Epoch 4/10
18/18 [==============================] - 0s 28ms/step - loss: 0.3998 - acc: 0.8306 - val_loss: 9.3075 - val_acc: 0.9750
Epoch 5/10
18/18 [==============================] - 0s 27ms/step - loss: 0.3408 - acc: 0.8361 - val_loss: 2.2064 - val_acc: 0.9750
Epoch 6/10
18/18 [==============================] - 0s 26ms/step - loss: 0.3269 - acc: 0.8750 - val_loss: 5.9478 - val_acc: 0.9750

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/10
18/18 [==============================


Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/10
18/18 [==============================] - 0s 27ms/step - loss: 0.3055 - acc: 0.8639 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/10
18/18 [==============================] - 0s 27ms/step - loss: 0.2670 - acc: 0.8917 - val_loss: 5.1415 - val_acc: 0.9750
Epoch 8/10
18/18 [==============================] - 0s 26ms/step - loss: 0.2550 - acc: 0.8944 - val_loss: 0.0000e+00 - val_acc: 1.0000

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 9/10
18/18 [==============================] - 0s 26ms/step - loss: 0.2423 - acc: 0.9056 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 10/10
18/18 [==============================] - 0s 27ms/step - loss: 0.2404 - acc: 0.9056 - val_loss: 0.0000e+00 - val_acc: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20

  

18/18 [==============================] - 0s 26ms/step - loss: 0.3223 - acc: 0.8667 - val_loss: 1.5366 - val_acc: 0.9750
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      0.95      0.97        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40

Kfold iteration 10/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
Epoch 1/10
18/18 [==============================] - 0s 26ms/step - loss: 0.6940 - acc: 0.6056 - val_loss: 32.0663 - val_acc: 0.8750
Epoch 2/10
18/18 [==============================] - 0s 26ms/step - loss: 0.5510 - acc: 0.7472 - val_loss: 27.9313 - val_acc: 0.8750
Epoch 3/10
18/18 [==============================] - 0s 27ms/step - loss: 0.4333 - acc: 0.7806 - val_loss: 46.6798 - val_acc: 0.9000
Epoch 4/10
18/18 [==============================] - 0s 27ms/step - loss: 

In [7]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 96.24%
